In [ ]:
## Random Forest

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import connectorx as cx
import math
import time
import shap
import seaborn as sns
import os
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
import mlflow
from mlflow.models import infer_signature
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split
import statistics
import xgboost as xgb
import plotly.express as px


In [4]:
database = 'isgpostgres'
user = 'postgres'
password = '123@NjMpTs'
host = '20.62.75.39'
port = '9030'
encoded_password = urllib.parse.quote(password)
connection_string = "postgresql+psycopg2://" + user + ":" + encoded_password + "@" + host + ":" + port + "/" + database
engine = create_engine(connection_string)

sql_command = """ select * from tm_new_data.prev_cur_model_data_240529 """

# Execute the query and fetch the data
data = pd.read_sql_query(sql_command, engine)

In [5]:
data['extract'] = list(map(lambda x : x[0] , data['prev_ncc_arr']))

In [58]:
data.shape

(275598, 30)

In [59]:
new_data = data.copy()

In [60]:
new_data.shape

(275598, 30)

In [61]:
new_data.columns

Index(['isg_osm_id', 'highway_id', 'region', 'roadwaytype_id', 'cur_lanes',
       'cur_lanes_cat', 'cur_maxspeed', 'cur_spd_cat', 'cur_final_place',
       'cur_areatype', 'cur_capacity', 'cur_cap_cat', 'prev_connected_count',
       'volume', 'prev_isg_osm_id_arr', 'prev_highway_id_arr',
       'prev_region_arr', 'prev_roadwaytype_id_arr', 'prev_lanes_arr',
       'prev_lanes_cat_arr', 'prev_maxspeed_arr', 'prev_spd_cat_arr',
       'prev_final_place_arr', 'prev_areatype_arr', 'prev_capacity_arr',
       'prev_cap_cat_arr', 'prev_volume_arr', 'prev_volume', 'prev_ncc_arr',
       'extract'],
      dtype='object')

In [66]:
selective_features_lst = [ 'highway_id', 'region', 'roadwaytype_id',
                          'cur_lanes_cat','cur_spd_cat','cur_cap_cat','prev_connected_count',
                           'prev_highway_id_arr', 'prev_region_arr',
                           'prev_roadwaytype_id_arr', 'prev_ncc_arr',
                           'prev_lanes_cat_arr', 'prev_spd_cat_arr',
                           'prev_cap_cat_arr', 'prev_final_place_arr',
                           'prev_areatype_arr',
                           'prev_volume','volume']

In [67]:
new_data_2 = data[selective_features_lst].copy()

In [68]:
new_data_2.isnull().sum()

highway_id                      0
region                          0
roadwaytype_id                  0
cur_lanes_cat                   0
cur_spd_cat                     0
cur_cap_cat                     0
prev_connected_count            0
prev_highway_id_arr             0
prev_region_arr                 0
prev_roadwaytype_id_arr         0
prev_ncc_arr                    0
prev_lanes_cat_arr              0
prev_spd_cat_arr                0
prev_cap_cat_arr                0
prev_final_place_arr            0
prev_areatype_arr               0
prev_volume                223651
volume                          0
dtype: int64

In [69]:
seen_df = new_data_2[(new_data_2['volume']>0 ) & (new_data_2['prev_volume']>0 ) ]
unseen_df = new_data_2[(new_data_2['volume'] < 0 ) & (new_data_2['prev_volume']>0 )]

In [134]:
seen_df.shape,unseen_df.shape

((37301, 18), (14646, 18))

In [135]:
seen_df.isnull().sum()

highway_id                 0
region                     0
roadwaytype_id             0
cur_lanes_cat              0
cur_spd_cat                0
cur_cap_cat                0
prev_connected_count       0
prev_highway_id_arr        0
prev_region_arr            0
prev_roadwaytype_id_arr    0
prev_ncc_arr               0
prev_lanes_cat_arr         0
prev_spd_cat_arr           0
prev_cap_cat_arr           0
prev_final_place_arr       0
prev_areatype_arr          0
prev_volume                0
volume                     0
dtype: int64

In [136]:
unseen_df.isnull().sum()

highway_id                 0
region                     0
roadwaytype_id             0
cur_lanes_cat              0
cur_spd_cat                0
cur_cap_cat                0
prev_connected_count       0
prev_highway_id_arr        0
prev_region_arr            0
prev_roadwaytype_id_arr    0
prev_ncc_arr               0
prev_lanes_cat_arr         0
prev_spd_cat_arr           0
prev_cap_cat_arr           0
prev_final_place_arr       0
prev_areatype_arr          0
prev_volume                0
volume                     0
dtype: int64

In [186]:
encoded_feature_lst = ['prev_highway_id_arr', 'prev_region_arr',
                           'prev_roadwaytype_id_arr', 'prev_ncc_arr',
                           'prev_lanes_cat_arr', 'prev_spd_cat_arr',
                           'prev_cap_cat_arr', 'prev_final_place_arr',
                           'prev_areatype_arr']

unencoded_feature_lst =  ['highway_id', 'region', 'roadwaytype_id','cur_lanes_cat',
                          'cur_spd_cat','cur_cap_cat','prev_connected_count','prev_volume']

target_feature_lst= ['volume']

In [187]:
features_unencoded_df = seen_df[unencoded_feature_lst].reset_index(drop=True)

features_unencoded_df.shape

(37301, 8)

In [188]:
features_unencoded_df.head()

,highway_id,region,roadwaytype_id,cur_lanes_cat,cur_spd_cat,cur_cap_cat,prev_connected_count,prev_volume
0,2,5,16,2,1,2,1,2948.0
1,1,5,10,2,3,3,2,2579.0
2,1,5,10,2,3,3,1,2579.0
3,1,5,10,2,3,3,1,2877.0
4,1,5,10,2,3,2,2,1139.0


In [191]:
mlb = MultiLabelBinarizer()

features_encoded_df = pd.DataFrame()
unseen_features_encoded_df = pd.DataFrame()

for col in encoded_feature_lst:
    #one_hot_encoded = mlb.fit_transform(seen_df[col])
    mlb.fit(seen_df[col])  # Fit the encoder
    
    seen_one_hot_encoded = mlb.transform(seen_df[col]) 
    seen_temp_df = pd.DataFrame(seen_one_hot_encoded, columns=[f'{col}_{i}' for i in range(1, seen_one_hot_encoded.shape[1] + 1)])
    features_encoded_df = pd.concat([features_encoded_df,seen_temp_df],axis=1)
    
    ## Unseen
    unseen_one_hot_encoded = mlb.transform(unseen_df[col])
    unsee_temp_df = pd.DataFrame(unseen_one_hot_encoded, columns=[f'{col}_{i}' for i in range(1, unseen_one_hot_encoded.shape[1] + 1)])
    unseen_features_encoded_df = pd.concat([unseen_features_encoded_df,unsee_temp_df],axis=1)
    
    
features_encoded_df.shape,unseen_features_encoded_df.shape

((37301, 59), (14646, 59))

In [192]:
features_encoded_df.head()

,prev_highway_id_arr_1,prev_highway_id_arr_2,prev_highway_id_arr_3,prev_highway_id_arr_4,prev_highway_id_arr_5,prev_highway_id_arr_6,prev_region_arr_1,prev_region_arr_2,prev_region_arr_3,prev_region_arr_4,...,prev_cap_cat_arr_4,prev_final_place_arr_1,prev_final_place_arr_2,prev_final_place_arr_3,prev_final_place_arr_4,prev_final_place_arr_5,prev_final_place_arr_6,prev_final_place_arr_7,prev_areatype_arr_1,prev_areatype_arr_2
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1
4,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [193]:
final_seen_df = pd.concat([features_unencoded_df,features_encoded_df],axis=1)
final_seen_df.shape

(37301, 67)

In [194]:
X,y = final_seen_df,seen_df[target_feature_lst]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape,X_test.shape, y_train.shape, y_test.shape

((29840, 67), (7461, 67), (29840, 1), (7461, 1))

In [203]:
def geh_check(row):
    current_vol = row['prev_volume']
    next_vol = row['predicted_volume']
    metric_value = float(np.sqrt(2 * np.sum((current_vol - next_vol) ** 2) / np.sum(current_vol + next_vol)))
    return metric_value

In [204]:
def mean_geh(y_test, y_pred):
    geh_lst = []
    for i,j in zip(y_test, y_pred):
        row = {'volume':i,'predicted_volume':j}
        geh_lst.append(geh_check(row))
    return statistics.mean(geh_lst)

In [205]:
def log_model_summary(xg_reg,X_train,X_test,y_train,y_test,y_pred):
    
    # Log Parameter
    xgb_params = xg_reg.get_params()
    mlflow.log_params(xgb_params)
    
    Test_Data_Df = X_test.copy()
    Test_Data_Df['volume'] = y_test['volume']
    Test_Data_Df['predicted_volume']=y_pred
    Test_Data_Df['predicted_volume']=Test_Data_Df['predicted_volume'].round().astype(int)
    Test_Data_Df['diff']=Test_Data_Df['volume']-Test_Data_Df['predicted_volume']
    Test_Data_Df['geh'] = Test_Data_Df.apply(lambda row: geh_check(row), axis=1)
    Test_Data_Df['geh'] = Test_Data_Df['geh'].round(2)
    
    # Log Metrics
    mse = round(mean_squared_error(y_test, y_pred),3)
    rmse = round(np.sqrt(mse),3)
    r2 = round(r2_score(y_test, y_pred),3)
    n = X_test.shape[0]
    p = X_test.shape[1]
    adjusted_r2score =  round((1 - (1 - r2) * (n - 1) / (n - p - 1)),3)
    mae = round(mean_absolute_error(y_test, y_pred),3)
    
    m_geh = round(mean_geh(y_test.values, y_pred),3)
    geh_grt_5 = Test_Data_Df[Test_Data_Df['geh']>5].shape[0]
    geh_error_per = round(((geh_grt_5/n)*100),3)
    
    metrics = {'mse':mse,'rmse':rmse,'r2score':r2,'mae':mae,'adjusted_r2score' : adjusted_r2score,
               'mean_geh':m_geh,'test_data_size':n,'geh_grt_5':geh_grt_5,'geh_error_per':geh_error_per}
    
    mlflow.log_metrics(metrics)
    
    
    # log tags
    tags = {'Model_Name' : 'RF',
            'Train Size' : X_train.shape[0],
            'Test Size' : X_test.shape[0],
            'No. Of Input Feature' : len(input_feature_lst),
            'No. Of Target Feature ': len(target_feature_lst),
            'input_feature_lst':input_feature_lst,
            'target_feature_lst': target_feature_lst}
    
    mlflow.set_tags(tags)

    return  Test_Data_Df

In [206]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'isgadmin'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'infosense12@'
tracking_server_uri = "https://ptt2-mlflow.isgsuite.com/"
mlflow.set_tracking_uri(tracking_server_uri)
tracking_uri = mlflow.get_tracking_uri()
print("Tracking URI:", tracking_uri)

Tracking URI: https://ptt2-mlflow.isgsuite.com/


In [207]:
# experiment_name = "TM_New_Approach"
# experiment = mlflow.set_experiment(experiment_name)
# experiment_id = experiment.experiment_id
run_name = 'RF_FC_1_2_3_Encoded'
description = 'previous as input values and current is target'
# Get Experiment Details
# print(f"Experiment_id: {experiment.experiment_id}")
print(f"Run Name: {run_name}")
print(f"Description: {description}")
# print(f"Artifact Location: {experiment.artifact_location}")

Run Name: RF_FC_1_2_3_Encoded
Description: previous as input values and current is target


In [208]:
with mlflow.start_run(experiment_id="59",run_name = run_name,description=description) as run:

    # Define the XGBoost regressor with the specified parameters
    params = {
    'n_estimators' : 500,
    'max_depth' : 8,
    'max_leaf_nodes' : 2000,
    'min_samples_split' : 2,
    'min_samples_leaf' : 15,
    'bootstrap' : False,
    'random_state' : 1234,
    'criterion' : 'poisson',
    'max_features' : 0.6
    }
    
    # Train the model
    xg_reg = RandomForestRegressor(**params)
    xg_reg.fit(X_train, y_train)
    # Make predictions on the test set
    y_pred = xg_reg.predict(X_test)
    y_test['volume'] = round(y_test['volume'],0).astype(int)
    y_pred = np.rint(y_pred).astype(int)
    
    # Log Model Summary
    Test_Data_Df = log_model_summary(xg_reg,X_train,X_test,y_train,y_test,y_pred)
    
    # Store Indexes
    # X_train_index = seen_df_1[seen_df_1.index.isin(X_train.index)].row_num.tolist()
    # X_test_index = seen_df_1[seen_df_1.index.isin(X_test.index)].row_num.tolist()
    # data_json = {'X_train_index': X_train_index,'X_test_index': X_test_index}
    
    # mlflow.log_dict(data_json, "Data_Index.json")
   
    
    # Log Feature Importance

    explainer = shap.TreeExplainer(xg_reg)
    shap_values = explainer.shap_values(X_train[:500])
    fig = shap.summary_plot(shap_values, X_train[:500], plot_type="bar",show=False)
    # Save the Image
    image_path = "Feature_Importance.png"
    plt.savefig(image_path)
    mlflow.log_artifact(image_path, "images")
    
#     Actual Vs Prediction Plot
    fig = px.scatter(Test_Data_Df, x='volume', y='predicted_volume')
    fig.add_trace(px.line(Test_Data_Df, x='volume', y='volume').data[0])
    fig.update_layout(xaxis_title='Actual Values', yaxis_title='Predicted Values')
    html_str = fig.to_html(full_html=False)
    with open("Actual_Vs_Prediction_Plot.html", "w") as f:
        f.write(html_str)
    mlflow.log_artifact("Actual_Vs_Prediction_Plot.html", "html")
        
# #     Actual And Prediction Distribution
    
#     fig, axs = plt.subplots(1, 2, figsize=(14, 6))
#     sns.histplot(Test_Data_Df['volume'], color='blue', kde=True, ax=axs[0])
#     axs[0].set_title('Actual Distribution')
#     axs[0].set_xlabel('Values')
#     axs[0].set_ylabel('Frequency')
#     sns.histplot(Test_Data_Df['predicted_volume'], color='orange', kde=True, ax=axs[1])
#     axs[1].set_title('Predicted Distribution')
#     axs[1].set_xlabel('Values')
#     axs[1].set_ylabel('Frequency')
#     plt.tight_layout()

#     image_path = "Actual_And_Prediction_Distribution.png"
#     plt.savefig(image_path)
#     mlflow.log_artifact(image_path, "images")
    
#     pd.options.mode.use_inf_as_null = True
# #     Residuals Plot
#     plt.figure(figsize=(8, 6))
#     sns.histplot(Test_Data_Df['diff'], bins=100, kde=False, color='skyblue')
#     plt.xlabel('Values')
#     plt.ylabel('Frequency')
#     plt.title('Histogram with KDE')
#     plt.xlim(-500, 500)
#     plt.grid(True)
    
#     image_path = "Residuals_Plot.png"
#     plt.savefig(image_path)
#     mlflow.log_artifact(image_path, "images")
    
#     Log Model
    signature = infer_signature(X_test, xg_reg.predict(X_test))
    mlflow.sklearn.log_model(xg_reg, "model", signature=signature)
    
    mlflow.end_run()  

C:\Users\VipulPrajapati\anaconda3\lib\site-packages\sklearn\base.py:1152: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



KeyError: 'prev_volume'

In [ ]:
Test_Data_Df[Test_Data_Df['geh']>5].shape,Test_Data_Df[Test_Data_Df['geh']>25].shape

### Prepare Data For unseen data

In [195]:
final_unseen_df = pd.concat([unseen_features_encoded_df,unseen_features_unencoded_df],axis=1)
final_unseen_df.shape

(14646, 67)

In [198]:
final_unseen_df.columns

Index(['prev_highway_id_arr_1', 'prev_highway_id_arr_2',
       'prev_highway_id_arr_3', 'prev_highway_id_arr_4',
       'prev_highway_id_arr_5', 'prev_highway_id_arr_6', 'prev_region_arr_1',
       'prev_region_arr_2', 'prev_region_arr_3', 'prev_region_arr_4',
       'prev_region_arr_5', 'prev_region_arr_6', 'prev_region_arr_7',
       'prev_region_arr_8', 'prev_region_arr_9', 'prev_region_arr_10',
       'prev_region_arr_11', 'prev_roadwaytype_id_arr_1',
       'prev_roadwaytype_id_arr_2', 'prev_roadwaytype_id_arr_3',
       'prev_roadwaytype_id_arr_4', 'prev_roadwaytype_id_arr_5',
       'prev_roadwaytype_id_arr_6', 'prev_roadwaytype_id_arr_7',
       'prev_roadwaytype_id_arr_8', 'prev_roadwaytype_id_arr_9',
       'prev_roadwaytype_id_arr_10', 'prev_roadwaytype_id_arr_11',
       'prev_roadwaytype_id_arr_12', 'prev_roadwaytype_id_arr_13',
       'prev_roadwaytype_id_arr_14', 'prev_roadwaytype_id_arr_15',
       'prev_roadwaytype_id_arr_16', 'prev_roadwaytype_id_arr_17',
       'pre

In [196]:
unseen_df_1 = final_unseen_df.copy()

In [201]:
unseen_pred = xg_reg.predict(final_unseen_df[final_seen_df.columns.tolist()])

unseen_df_1['predicted_volume'] = unseen_pred
unseen_df_1['predicted_volume'] = unseen_df_1['predicted_volume'].round().astype(int)

In [202]:

unseen_df_1['GEH'] = unseen_df_1.apply(geh_check, axis=1)
unseen_df_1.shape

KeyError: 'volume'